# W261 Final Project ETL for Development Sample

## Notebook Set-Up

In [1]:
# imports
import time
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from pyspark.sql import Row
from pyspark.ml.feature import CountVectorizer
from pyspark.sql import DataFrame

In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
# store path to notebook
PWD = !pwd
PWD = PWD[0]
PWD

'/media/notebooks'

In [4]:
# start Spark Session
from pyspark.sql import SparkSession
app_name = "w261FinalProject"
master = "local[*]"
spark = SparkSession\
        .builder\
        .appName(app_name)\
        .master(master)\
        .getOrCreate()
sc = spark.sparkContext

__`NOTE:`__ Monitor the progress of your jobs using the Spark UI at: http://localhost:4040/jobs/

## Pre-Processing

### Load Data

In [5]:
original_trainRDD = sc.textFile('data/train.txt')
original_testRDD = sc.textFile('data/test.txt')

### Sample Data

In [6]:
#change the seed for a different sample
sampleRDD1, sampleRDD2 = original_trainRDD.randomSplit([0.99995,0.00005], seed = 1)
sampleRDD2.cache()

PythonRDD[4] at RDD at PythonRDD.scala:49

In [7]:
ncol = len(sampleRDD2.take(1)[0].split('\t'))
nrow = sampleRDD2.count()
print(f'This sample containts {nrow} rows and {ncol} columns')

This sample containts 2248 rows and 40 columns


In [8]:
def avgFeatures(line):
    
    count = 0
    feats = line.split('\t')[1:]
    
    for feat in feats:
        if feat != '':
            count += 1

    return count

print("There is an average of", str(round(sampleRDD2.map(avgFeatures).mean(),2)), "populated features per observation.")

There is an average of 33.52 populated features per observation.


### Normalize Data

### Dimension Reduction

### Transformation

In [9]:
# function to parse raw data and tag feature values with type and feature indices
def parseCV(line):
    """
    Map record_csv_string --> (features, label)
    """

    # start of categorical features
    col_start = 14
    
    raw_values = line.split('\t')
    label = int(raw_values[0])
    
    # parse numeric features
    numericals = []
    for idx, value in enumerate(raw_values[1:col_start]):
        if value != '':
            numericals.append('n' + str(idx) + '_' + str(value))
            
    # parse categorical features
    categories = []
    for idx, value in enumerate(raw_values[col_start:]):
        if value != '':
            categories.append('c'+ str(idx) + '_' + str(value))

    return Row(label=label, raw=numericals + categories)

# function to one hot encode all features using a count vectorizer
def vectorizeCV(DF):
    
    vectorizer = CountVectorizer()
    cv = CountVectorizer(inputCol="raw", outputCol="features")
    
    model = cv.fit(DF)
    result = model.transform(DF)
    
    return result

# call functions
parsedDF = sampleRDD2.map(parseCV).toDF().cache()
vectorizedDF = vectorizeCV(parsedDF)

In [10]:
vectorizedDF.show(truncate=True)

+-----+--------------------+--------------------+
|label|                 raw|            features|
+-----+--------------------+--------------------+
|    0|[n1_4, n2_50, n3_...|(18545,[0,1,2,4,5...|
|    0|[n1_12, n2_20, n3...|(18545,[0,1,2,5,1...|
|    0|[n0_8, n1_17, n3_...|(18545,[0,1,4,12,...|
|    0|[n0_0, n1_144, n4...|(18545,[0,2,3,4,5...|
|    0|[n1_0, n2_5, n4_3...|(18545,[0,2,3,6,1...|
|    0|[n1_323, n2_2, n3...|(18545,[1,2,14,16...|
|    0|[n0_0, n1_424, n3...|(18545,[0,1,2,4,6...|
|    0|[n0_0, n1_13, n2_...|(18545,[0,1,2,5,6...|
|    0|[n1_180, n2_6, n3...|(18545,[1,2,8,14,...|
|    0|[n1_21, n2_3, n3_...|(18545,[1,2,6,10,...|
|    0|[n1_3, n2_7, n3_1...|(18545,[0,1,2,5,8...|
|    0|[n1_1, n2_2, n4_3...|(18545,[0,5,9,14,...|
|    0|[n0_0, n1_2, n4_3...|(18545,[0,2,5,6,1...|
|    1|[n1_-1, n4_26295,...|(18545,[0,2,5,9,2...|
|    0|[n0_0, n1_0, n2_1...|(18545,[0,1,5,7,9...|
|    1|[n0_1, n1_196, n3...|(18545,[0,1,2,3,5...|
|    0|[n0_0, n1_-1, n4_...|(18545,[0,1,3,4,5...|


In [11]:
vectorizedRDD = vectorizedDF.select(['label', 'features']).rdd.cache()

In [12]:
num_feats = vectorizedRDD.take(1)[0][1].size
percent_pos = vectorizedRDD.map(lambda x: x[0]).mean()

print("Number of total expanded features:", num_feats)
print("Percent of data in the positive class:", percent_pos)

Number of total expanded features: 18545
Percent of data in the positive class: 0.254003558718861


## Exploratory Data Analysis

## Modeling

In [13]:
def predict_grad(pair, k_br, b_br, w_br, V_br):
    """
        Compute the predicted probability AND return the gradient (?)
        Args:
            pair - records are in (label, sparse feature set) format
        Broadcast:
            b - bias term (scalar)
            w - linear weight vector (array)
            k - number of factors (def=2)
            V - factor matrix of size (d dimensions, k=2 factors)
        Returns:
            predRDD - pair of ([label, predicted probability], [set of weight vectors in csr_matrix format])
    """
    
    label = pair[0]
    feats = pair[1]
    
    # start with linear weight dot product
    linear_sum = np.dot(w_br.value[0][feats.indices], feats.values)

    # factor matrix interaction sum
    factor_sum = 0.0
    lh_factor = [0.0]*k_br.value
    rh_factor = [0.0]*k_br.value
    
    for f in range(0, k_br.value):
        lh_factor[f] = np.dot(V_br.value[f][feats.indices], feats.values)  #KEY--this is used in v_grad matrix below
        rh_factor[f] = np.dot(V_br.value[f][feats.indices]**2, feats.values**2)
        factor_sum += (lh_factor[f]**2 - rh_factor[f])
    factor_sum = 0.5 * factor_sum
    
    pre_prob = b_br.value + linear_sum + factor_sum
    
    prob = 1.0 / (1 + np.exp(-pre_prob))  #logit transformation
    
    #compute Gradients
    b_grad = prob - label
    
    w_grad = csr_matrix((b_grad*feats.values, feats.indices, np.array([0, feats.indices.size])), (1, w_br.value.shape[1]))
    
    v_grad = csr_matrix((V_br.value.shape[0], V_br.value.shape[1]))
    for f in range(0, k_br.value):  # WORKING
        for i in feats.indices:
            i = int(i)
            v_grad[(f,i)] = b_grad * (feats[i]*lh_factor[f] - V_br.value[f][i]*(feats[i]**2))
    
    
    return ([label, prob], [b_grad, w_grad, v_grad])

In [14]:
def logLoss(pair):
    """parallelize log loss"""
    y = pair[0][0]
    
    eps = 1.0e-16
    if pair[0][1] == 0:
        y_hat = eps
    elif pair[0][1] == 1:
        y_hat = 1-eps
    else:
        y_hat = pair[0][1]
    
    return -(y * np.log(y_hat) + (1-y) * np.log(1-y_hat))


In [23]:
def iterateSGD(dataRDD, k, bInit, wInit, vInit, nIter = 2, learningRate = 0.1, useReg = False, regParam = 0.001):

    k_br = sc.broadcast(k)    
    b_br = sc.broadcast(bInit)
    w_br = sc.broadcast(wInit)
    V_br = sc.broadcast(vInit)

    losses = []

    for i in range(nIter):
        print('-' * 25 + 'Iteration ' + str(i) + '-' * 25)
        predRDD = dataRDD.map(lambda x: predict_grad(x, k_br, b_br, w_br, V_br)).cache()
        loss = predRDD.map(logLoss).mean() + \
                int(useReg)*(regParam/2)*(np.linalg.norm(w_br.value)**2 + np.linalg.norm(V_br.value)**2)
        losses.append(loss)
        print(f'Current log-loss: {loss}')

        # calculate average gradient for b
        bGrad = predRDD.map(lambda x: x[1][0]).mean()
        print(f"Bias: {bGrad}")

        # calculate average gradient for w
        N = predRDD.count()
        wGrad = (1/N) * predRDD.map(lambda x: x[1][1]).reduce(lambda a,b: a+b)
        print(f"wGrad shape: {wGrad.shape}")

        # calculate average gradient for V
        vGrad = (1/N) * predRDD.map(lambda x: x[1][2]).reduce(lambda a,b: a+b)
        print(f"vGrad shape: {vGrad.shape}")

        ############## update weights ##############
        # first, unpersist broadcasts
        #predRDD.unpersist()
        b_br.unpersist()
        w_br.unpersist()
        V_br.unpersist()

        # update
        b_br = sc.broadcast(b_br.value - learningRate * bGrad)
        w_br = sc.broadcast(w_br.value - learningRate * (wGrad.toarray()+int(useReg)*regParam*np.linalg.norm(w_br.value)))
        V_br = sc.broadcast(V_br.value - learningRate * (vGrad.toarray()+int(useReg)*regParam*np.linalg.norm(V_br.value)))

In [24]:
# initialize weights
np.random.seed(24)
k = 2
b = 0.0
w = np.random.normal(0.0, 0.02, (1, num_feats))
V = np.random.normal(0.0, 0.02, (k, num_feats))

nIter = 5
start = time.time()
iterateSGD(vectorizedRDD, k, b, w, V, nIter, learningRate = 0.1, useReg = False)
print(f'Performed {nIter} iterations in {time.time() - start} seconds')

-------------------------Iteration 0-------------------------
Current log-loss: 0.700875764695785
Bias: 0.25197560629662874
wGrad shape: (1, 18545)
vGrad shape: (2, 18545)
-------------------------Iteration 1-------------------------
Current log-loss: 0.6694363241610701
Bias: 0.219534603584837
wGrad shape: (1, 18545)
vGrad shape: (2, 18545)
-------------------------Iteration 2-------------------------
Current log-loss: 0.645515410213653
Bias: 0.19157597446487426
wGrad shape: (1, 18545)
vGrad shape: (2, 18545)
-------------------------Iteration 3-------------------------
Current log-loss: 0.6272102438813704
Bias: 0.16757782040609945
wGrad shape: (1, 18545)
vGrad shape: (2, 18545)
-------------------------Iteration 4-------------------------
Current log-loss: 0.6130943228628224
Bias: 0.14700050665793155
wGrad shape: (1, 18545)
vGrad shape: (2, 18545)
Performed 5 iterations in 419.2542371749878 seconds
